In [1]:
import Encoder
import Decoder
import GNN_LSTM
from MQRNN import MQRNN
from data import MQRNN_dataset, read_df, read_df2, read_df3,loss_result
import pandas as pd
import numpy as np
from data import MQRNN_dataset
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch
import random
from train import train
from sklearn.preprocessing import StandardScaler
from GNN_func import build_and_solve_milp, parse_mps_and_build_graphs
#from NEW_GNN0421 import build_graph_from_mps_sol
from Student_model import *
import pickle
import os
import time
import math

In [2]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import read
import numpy as np
import re
from collections import defaultdict
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch_geometric.nn as geom_nn

In [3]:
# ====================== 参数设置 ======================
price_data = pd.read_excel('./32SKU/price_32.xlsx')
target_width = price_data.shape[0]
SKUS = []
for i in range(target_width):
    SKUS.append(price_data.iloc[i][0])
prices = {}
for i in range(target_width):
    prices[SKUS[i]] = price_data.iloc[i][1]
# SKUS = ['EAEZAIN9501', 'EAEZAIN9501EZ3', 'NAEZAIN9501',
#         'NAEZAIN9501EZ', 'NAEZAIN9501EZ2', 'NAEZAIN9501EZ3',
#         'NAEZAIN9501EZ4', 'NAEZAIN9502']

sku_index = {sku: i for i, sku in enumerate(SKUS)}
TOTAL_PERIODS = 99
window_size = 7
num_windows = TOTAL_PERIODS - window_size + 1  # 滑动窗口的数量
TRAIN_RATIO = 0.8  # 训练集80%
np.random.seed(42)
torch.manual_seed(42)

# 加载Excel数据
# df_demand = pd.read_excel('demand_0206.xlsx')
# df_supply = pd.read_excel('supply_0206.xlsx')
# df_inventory = pd.read_excel('inventory_0206.xlsx')

#32个sku
df_demand = pd.read_excel('./32SKU/demand_top32.xlsx')
df_supply = pd.read_excel('./32SKU/supply_top32.xlsx')
df_inventory = pd.read_excel('./32SKU/inventory_top32.xlsx')

# 供给量
supply = df_supply.set_index('ID').to_dict(orient='index')
# 销售量
demand = df_demand.set_index('ID').to_dict(orient='index')
# 库存日报
inventory = df_inventory.set_index('ID').to_dict(orient='index')

# 成本参数设置
# prices = {
#      'EAEZAIN9501': 65, 'EAEZAIN9501EZ3': 70, 'NAEZAIN9501': 69,
#      'NAEZAIN9501EZ': 84, 'NAEZAIN9501EZ2': 89, 'NAEZAIN9501EZ3': 74,
#      'NAEZAIN9501EZ4': 79, 'NAEZAIN9502': 74
# }

# 计算各项成本
trans_cost = {}
for s1 in SKUS:
    for s2 in SKUS:
        if s1 != s2:
            # 改造成本 = 价格差的20%/6
            trans_cost[(s1, s2)] = 1.5

holding_cost_factor = 0.068
shortage_cost_factor = 0.3
M = 1e5
# 库存持有成本和缺货成本
holding_cost = {s: prices[s] * holding_cost_factor for s in SKUS}
shortage_cost = {s: prices[s] * shortage_cost_factor for s in SKUS}
# 输出目录准备
mps_dir = "mps_files_32SKU"
os.makedirs(mps_dir, exist_ok=True)

# ====================== 1. MILP建模与求解 ======================
for w in range(num_windows):
    print(f"正在处理周期 {w+1}/{num_windows}...")

    model = gp.Model(f"SKU_Transformation_Week_{w}")
    start_day = w
    end_day = w + window_size

    # 初始库存为前一天的库存
    if start_day == 0:
        current_inv = {s: df_inventory.loc[0, s] for s in SKUS}
    else:
        current_inv = {s: df_inventory.loc[start_day - 1, s] for s in SKUS}

    # ----------------------------
    # 定义变量
    # ----------------------------
    weekly_trans_vars = model.addVars(
        [(s1, s2) for s1 in SKUS for s2 in SKUS if s1 != s2 ],
        name="Weekly_Trans", vtype=GRB.INTEGER)
    for (s1, s2), var in weekly_trans_vars.items():
        var.VarName = f"Weekly_Trans_{s1}_{s2}_w{w}"
    inv_vars = model.addVars(SKUS, name="Inv", vtype=GRB.INTEGER ,lb=0)
    short_vars = model.addVars(SKUS, name="Short", vtype=GRB.INTEGER ,lb=0)
    # delta_vars = model.addVars(SKUS, name="IsShort", vtype=GRB.BINARY)
    # ----------------------------
    # 约束1：库存平衡
    # ----------------------------
    weekly_supply = {s: 0 for s in SKUS}
    weekly_demand = {s: 0 for s in SKUS}
    for d in range(window_size):
        day = start_day + d
        idx = w * window_size + d
        for s in SKUS:
            weekly_supply[s] += df_supply.loc[day, s]
            weekly_demand[s] += df_demand.loc[day, s]

    for s in SKUS:
        trans_in = gp.quicksum(weekly_trans_vars[s2, s] for s2 in SKUS if s2 != s)
        trans_out = gp.quicksum(weekly_trans_vars[s, s2] for s2 in SKUS if s2 != s)

        # TS_s= (current_inv[s] + weekly_supply[s] + trans_in - trans_out)
        # # 库存下限
        # model.addConstr(inv_vars[s] >= TS_s - weekly_demand[s])
        # model.addConstr(inv_vars[s] <= (TS_s - weekly_demand[s]) + M * (1 - delta_vars[s]))
        # model.addConstr(short_vars[s] >= weekly_demand[s] - TS_s)
        # model.addConstr(short_vars[s] <= (weekly_demand[s] - TS_s) + M * delta_vars[s])

        model.addConstr(
            current_inv[s] + weekly_supply[s] + trans_in + short_vars[s] ==
            weekly_demand[s] + inv_vars[s] + trans_out,
            name=f"WeeklyBalance_{s}_{w}"
        )
    # ----------------------------
    # 约束2：缺货不能为负
    # ----------------------------
    model.addConstrs((short_vars[s] >= 0 for s in SKUS), name="Shortage_NonNegative")
    # ----------------------------
    # 约束3：高配不能转低配
    # ----------------------------
    for s1 in SKUS:
        for s2 in SKUS:
            if sku_index[s1] > sku_index[s2]:
                model.addConstr(weekly_trans_vars[s1, s2] == 0, name=f"No_Downgrade_{s1}_{s2}_{w}")
    # ----------------------------
    # 目标函数
    # ----------------------------
    revenue = gp.quicksum(
        prices[s] * (weekly_demand[s] - short_vars[s]) for s in SKUS
    )
    cost = (
        gp.quicksum(weekly_trans_vars[s1, s2] * trans_cost[s1, s2]
                    for s1 in SKUS for s2 in SKUS if s1 != s2) +
        gp.quicksum(inv_vars[s] * holding_cost[s] for s in SKUS ) +
        gp.quicksum(short_vars[s] * shortage_cost[s] for s in SKUS )
    )
    model.setObjective(revenue - cost, GRB.MAXIMIZE)

    # ----------------------------
    # 模型优化与导出
    # ----------------------------
    model.setParam('OutputFlag', 0)
    model.write(f"{mps_dir}/MILP_week_{w}.mps")
    model.optimize()
    model.write(f"{mps_dir}/MILP_week_{w}.sol")  # 保存解文件

    print(f"MILP solved successfully. Obj: {model.ObjVal:.2f}")
    # 获取所有变量值
    var_dict = {v.VarName: v.X for v in model.getVars()}

torch.manual_seed(42)
np.random.seed(42)

MPS_DIR = f"{mps_dir}/MILP_week_{w}.mps"
SOL_DIR = f"{mps_dir}/MILP_week_{w}.sol"

# 2. 解析 MPS 文件函数
def parse_mps_file(file_path):
    model = read(MPS_DIR)
    variables = [var.VarName for var in model.getVars()]
    constraints = [con.ConstrName for con in model.getConstrs()]
    edges = []
    for i, var in enumerate(variables):
        for j, con in enumerate(constraints):
            coef = np.random.choice([0, 1], p=[0.7, 0.3]) * np.random.uniform(0.1, 1.0)
            if coef != 0:
                edges.append((i, j, coef))
    return variables, constraints, edges

# 3. 解析 SOL 文件函数
def parse_sol_file(file_path,week_idx):
    variable_values = {}
    obj_value = None
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            # 获取目标函数值
            if line.startswith("# Objective value"):
                obj_value = float(line.split('=')[1].strip())

            if not line or line.startswith('#'):
                continue

            parts = line.split()
            if len(parts) != 2:
                continue
            var_name, value_str = parts
            try:
                value = float(value_str)
                variable_values[var_name] = value
            except ValueError:
                continue

    return variable_values, obj_value

# 4. 构建 PyG 图结构
def safe_normalize_graph_index(x, edge_index):
    nodes_used = torch.unique(edge_index)
    id_map = {old_id.item(): new_id for new_id, old_id in enumerate(nodes_used)}
    new_edge_index = torch.stack([
        torch.tensor([id_map[i.item()] for i in edge_index[0]]),
        torch.tensor([id_map[i.item()] for i in edge_index[1]])
    ], dim=0)
    new_x = x[nodes_used]

    return new_x, new_edge_index

def build_graph_from_mps_sol(mps_path, sol_path,period_id):
    variables, constraints, edges = parse_mps_file(mps_path)
    varname_to_idx = {var: idx for idx, var in enumerate(variables)}
    values, obj = parse_sol_file(sol_path, period_id)
    # print(f"[DEBUG] Week {period_id} parsed {len(values)} variables, example: {list(values.items())[:5]}")

    num_vars = len(variables)
    num_cons = len(constraints)
    num_nodes = num_vars + num_cons

    edge_index = [[], []]
    edge_attr = []

    for var_idx, con_idx, coef in edges:
        edge_index[0].append(var_idx)
        edge_index[1].append(num_vars + con_idx)
        edge_attr.append([coef])
        edge_index[0].append(num_vars + con_idx)
        edge_index[1].append(var_idx)
        edge_attr.append([coef])

    x = torch.eye(num_nodes, dtype=torch.float)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)
    y_values = [0.0] * num_vars

    # 获取SKU——pairs
    for var_name, value in values.items():
        if var_name in varname_to_idx:
            idx = varname_to_idx[var_name]
            y_values[idx] = value
    y_tensor = torch.tensor(y_values, dtype=torch.float)

    var_names = variables
    var_names += [''] * num_cons

    data = Data(
        x=x,
        edge_index=edge_index,
        edge_attr=edge_attr,
        y=y_tensor,
    )
    data.period_id = torch.tensor([period_id], dtype=torch.long)
    data.milp_obj = obj
    data.var_names = var_names
    return data

# for test （可删）
# for w in range(num_windows):
#     MPS_DIR = f"{mps_dir}/MILP_week_{w}.mps"
#     SOL_DIR = f"{mps_dir}/MILP_week_{w}.sol"
#     print(build_graph_from_mps_sol(MPS_DIR, SOL_DIR,w))

In [4]:
# 2. 解析 MPS 文件函数
def parse_mps_file(file_path):
    model = read(MPS_DIR)
    variables = [var.VarName for var in model.getVars()]
    constraints = [con.ConstrName for con in model.getConstrs()]
    edges = []
    for i, var in enumerate(variables):
        for j, con in enumerate(constraints):
            coef = np.random.choice([0, 1], p=[0.7, 0.3]) * np.random.uniform(0.1, 1.0)
            if coef != 0:
                edges.append((i, j, coef))
    return variables, constraints, edges


# 3. 解析 SOL 文件函数
def parse_sol_file(file_path,week_idx):
    variable_values = {}
    obj_value = None
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            # 获取目标函数值
            if line.startswith("# Objective value"):
                obj_value = float(line.split('=')[1].strip())

            if not line or line.startswith('#'):
                continue

            parts = line.split()
            if len(parts) != 2:
                continue
            var_name, value_str = parts
            try:
                value = float(value_str)
                variable_values[var_name] = value
            except ValueError:
                continue

    return variable_values, obj_value


def build_graph_from_mps_sol(mps_path, sol_path,period_id):
    variables, constraints, edges = parse_mps_file(mps_path)
    varname_to_idx = {var: idx for idx, var in enumerate(variables)}
    values, obj = parse_sol_file(sol_path, period_id)
    # print(f"[DEBUG] Week {period_id} parsed {len(values)} variables, example: {list(values.items())[:5]}")

    num_vars = len(variables)
    num_cons = len(constraints)
    num_nodes = num_vars + num_cons

    edge_index = [[], []]
    edge_attr = []

    for var_idx, con_idx, coef in edges:
        edge_index[0].append(var_idx)
        edge_index[1].append(num_vars + con_idx)
        edge_attr.append([coef])
        edge_index[0].append(num_vars + con_idx)
        edge_index[1].append(var_idx)
        edge_attr.append([coef])

    x = torch.eye(num_nodes, dtype=torch.float)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)
    y_values = [0.0] * num_vars

    # 获取SKU——pairs
    for var_name, value in values.items():
        if var_name in varname_to_idx:
            idx = varname_to_idx[var_name]
            y_values[idx] = value
    y_tensor = torch.tensor(y_values, dtype=torch.float)

    var_names = variables
    var_names += [''] * num_cons

    data = Data(
        x=x,
        edge_index=edge_index,
        edge_attr=edge_attr,
        y=y_tensor,
    )
    data.period_id = torch.tensor([period_id], dtype=torch.long)
    data.milp_obj = obj
    data.var_names = var_names
    return data

In [5]:
# 5. 构建数据集
data_list = []

# 定义缓存文件路径
CACHE_FILE = f"{mps_dir}/data_list_cache.pkl"

# 检查缓存是否存在
if os.path.exists(CACHE_FILE):
    print("从缓存加载数据...")
    with open(CACHE_FILE, 'rb') as f:
        data_list = pickle.load(f)
else:
    for i in range(93):
        MPS_DIR = f"{mps_dir}/MILP_week_{i}.mps"
        SOL_DIR = f"{mps_dir}/MILP_week_{i}.sol"
        graph_data = build_graph_from_mps_sol(MPS_DIR, SOL_DIR,period_id=i)
        data_list.append(graph_data)

        with open(CACHE_FILE, 'wb') as f:
                pickle.dump(data_list, f)

从缓存加载数据...


In [6]:
data_list

[Data(x=[1616, 1616], edge_index=[2, 354550], edge_attr=[354550, 1], y=[1056], period_id=[1], milp_obj=63219.87199999999, var_names=[1616]),
 Data(x=[1616, 1616], edge_index=[2, 354008], edge_attr=[354008, 1], y=[1056], period_id=[1], milp_obj=95376.532, var_names=[1616]),
 Data(x=[1616, 1616], edge_index=[2, 354350], edge_attr=[354350, 1], y=[1056], period_id=[1], milp_obj=124994.224, var_names=[1616]),
 Data(x=[1616, 1616], edge_index=[2, 355462], edge_attr=[355462, 1], y=[1056], period_id=[1], milp_obj=137750.124, var_names=[1616]),
 Data(x=[1616, 1616], edge_index=[2, 354984], edge_attr=[354984, 1], y=[1056], period_id=[1], milp_obj=157916.772, var_names=[1616]),
 Data(x=[1616, 1616], edge_index=[2, 354470], edge_attr=[354470, 1], y=[1056], period_id=[1], milp_obj=173612.596, var_names=[1616]),
 Data(x=[1616, 1616], edge_index=[2, 355910], edge_attr=[355910, 1], y=[1056], period_id=[1], milp_obj=150065.152, var_names=[1616]),
 Data(x=[1616, 1616], edge_index=[2, 355512], edge_attr=

In [7]:
#prices

data_list[0].x.shape[0]

len(data_list)

In [8]:
# price_data = pd.read_excel('./32SKU/price_32.xlsx')
# target_width = price_data.shape[0]
# SKUS = []
# for i in range(target_width):
#     SKUS.append(price_data.iloc[i][0])
# prices = {}
# for i in range(target_width):
#     prices[SKUS[i]] = price_data.iloc[i][1]

In [9]:
#number_array = [1]
number = 1
loss_array_total = np.zeros(30)
predict = np.zeros(28)

In [10]:
config = {
        'horizon_size':3,
        'hidden_size':128,
        'quantiles': [0.05,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95],
        'columns': ['value'],
        'dropout': 0.1,
        'b_direction':False,
        'lr': 1e-4,
        'batch_size': 1,
        'num_epochs':10,
        'context_size': 16,
        'name': number,
    }
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#target_width = 8

In [11]:
data_df = pd.read_excel('./32SKU/input_32_half.xlsx', index_col='report_date')
#data_df = pd.read_excel('./input_1.xlsx', index_col='report_date')

In [12]:
#data_df

#二分图计算参数准备
SKUS = ['EAEZAIN9501', 'EAEZAIN9501EZ3', 'NAEZAIN9501',
        'NAEZAIN9501EZ', 'NAEZAIN9501EZ2', 'NAEZAIN9501EZ3',
        'NAEZAIN9501EZ4', 'NAEZAIN9502']
TOTAL_PERIODS = 184
TRAIN_RATIO = 0.8  # 154个训练周期，30个测试周期
np.random.seed(42)
torch.manual_seed(42)

# 加载Excel数据
#8个sku
df_demand = pd.read_excel('demand_0206.xlsx')
df_supply = pd.read_excel('supply_0206.xlsx')
df_inventory = pd.read_excel('inventory_0206.xlsx')
#32个sku
# df_demand = pd.read_excel('./32SKU/demand_top32.xlsx')
# df_supply = pd.read_excel('./32SKU/supply_top32.xlsx')
# df_inventory = pd.read_excel('./32SKU/inventory_top32.xlsx')

# 供给量
supply = df_supply.set_index('ID').to_dict(orient='index')
# 销售量
demand = df_demand.set_index('ID').to_dict(orient='index')
# 库存日报
#initial_inv = df_inventory.set_index('ID').to_dict(orient='index')

# 成本参数设置
#8个sku
prices = {
    'EAEZAIN9501': 65, 'EAEZAIN9501EZ3': 70, 'NAEZAIN9501': 69,
    'NAEZAIN9501EZ': 84, 'NAEZAIN9501EZ2': 89, 'NAEZAIN9501EZ3': 74,
    'NAEZAIN9501EZ4': 79, 'NAEZAIN9502': 74
}

# 计算各项成本
trans_cost = {}
for s1 in SKUS:
    for s2 in SKUS:
        if s1 != s2:
            # 改造成本 = 价格差的20%/6
            trans_cost[(s1, s2)] = abs(prices[s1] - prices[s2]) * 0.2 / 6

holding_cost_factor = 0.068
shortage_cost_factor = 0.3
# 库存持有成本和缺货成本
holding_cost = {s: prices[s] * holding_cost_factor for s in SKUS}
shortage_cost = {s: prices[s] * shortage_cost_factor for s in SKUS}

# 1. 构建并求解MILP
var_values = build_and_solve_milp()
# 2. 解析MPS并构建图数据集
GNN_dataset = parse_mps_and_build_graphs(var_values)

In [13]:
#GNN_dataset[5]

In [14]:
#len(GNN_dataset)

In [15]:
GNN_dataset = data_list

In [16]:
type(GNN_dataset)

list

In [17]:
len(GNN_dataset)

93

In [18]:
#target_width = 32
number = config['name']
horizon_size = config['horizon_size'] 
time_range = pd.DatetimeIndex(data_df.index)
#提取target数据
series_dict_target = {}
#series_dict[0] = data_df['value']
i = 0
for col in data_df.columns:
    if i < ((target_width - 1) * target_width):
        series_dict_target[i] = data_df[col]
    i += 1
    if i > (target_width - 1) * target_width + target_width: break
target_df = pd.DataFrame(index = time_range,data=series_dict_target)

#提取demand数据
series_dict_demand = {}
#series_dict[0] = data_df['value']
i = 0
for col in data_df.columns:
    if i >= ((target_width - 1) * target_width) and i < (target_width * target_width):
        series_dict_demand[i - ((target_width - 1) * target_width)] = data_df[col]
    i+=1
    if i >  (target_width + 2) * target_width + 1 : break
demand_df = pd.DataFrame(index=time_range,data=series_dict_demand)

#提取supply数据
series_dict_supply = {}
i = 0
for col in data_df.columns:
    if i >= (target_width * target_width) and i < ((target_width + 1) * target_width):
        series_dict_supply[i - (target_width * target_width)] = data_df[col]
    i += 1
    if i > (target_width + 2) * target_width + 1: break
supply_df = pd.DataFrame(index=time_range,data=series_dict_supply)

#提取inventory数据
series_dict_inventory = {}
i = 0
for col in data_df.columns:
    if i >= ((target_width + 1) * target_width) and i < ((target_width + 2) * target_width):
        series_dict_inventory[i - ((target_width + 1) * target_width)] = data_df[col]
    i += 1
    if i > (target_width + 2) * target_width + 1: break
inventory_df = pd.DataFrame(index=time_range,data=series_dict_inventory)

#提取特征数据
series_dict_covariate = {}
i = 0
for col in data_df.columns:
    if i >= ((target_width + 2) * target_width):
        series_dict_covariate[i - ((target_width + 2) * target_width)] = data_df[col]
    i += 1
covariate_df = pd.DataFrame(index=time_range,data=series_dict_covariate)

In [19]:
#covariate_df

In [20]:
n = int(20 / horizon_size)
train_target_df = target_df.iloc[horizon_size:-horizon_size * n]
test_target_df = target_df.iloc[-horizon_size * n:]

train_demand_df = demand_df.iloc[horizon_size:-horizon_size * n]
test_demand_df = demand_df.iloc[-horizon_size * n:]

train_supply_df = supply_df.iloc[horizon_size:-horizon_size * n]
test_supply_df = supply_df.iloc[-horizon_size * n:]

train_inventory_df = inventory_df.iloc[horizon_size:-horizon_size * n]
test_inventory_df = inventory_df.iloc[-horizon_size * n:]

train_covariate_df = covariate_df.iloc[:-(n + 1) * horizon_size, :]
test_covariate_df = covariate_df.iloc[-(n + 1) * horizon_size:-horizon_size, :]

train_GNN_dataset = GNN_dataset[:len(train_covariate_df)-horizon_size]

In [21]:
#train_demand_df

In [22]:
#train_supply_df

In [23]:
#train_inventory_df

In [24]:
#test_inventory_df

In [25]:
#train_covariate_df

In [26]:
#此部分用于生成供蒸馏训练的测试集和训练集
test_target_df = target_df
test_demand_df = demand_df
test_supply_df = supply_df
test_inventory_df = inventory_df
test_covariate_df = covariate_df

In [27]:
print(train_target_df.shape,test_target_df.shape)

(78, 992) (99, 992)


In [28]:
config['covariate_size'] = train_covariate_df.shape[1]
config['device'] = device
config['hidden_size'] = 512
config['dropout'] = 0.2
config['layer_size'] = 10
config['lr'] = 7e-4
config['context_size'] = 64
config['num_epochs'] = 100
horizon_size = config['horizon_size']
hidden_size = config['hidden_size']
quantiles = config['quantiles']
quantiles_size = len(quantiles)
columns = config['columns']
dropout = config['dropout']
layer_size = config['layer_size']
b_direction = config['b_direction']
lr = config['lr']
batch_size= config['batch_size']
num_epochs = config['num_epochs']
context_size = config['context_size']
covariate_size = config['covariate_size']
price = list(prices.values())
#target_width = 32
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [29]:
#a_ij表示转化关系，第 1 列 A_to_B = 1 表示A可转化为B，第2列 B_to_A = 0表示B不可转化为A
a_ij = []
for m in range(target_width):
    for n in range(target_width):
        if n == m:
            continue
        else:
            if price[m] >= price[n]:
                a_ij.append(1)
            else:a_ij.append(0)
n = int(20 / horizon_size)
#a_ij = [1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0]
#print(a_ij)
A = []
B = []

#约束添加 w_ij + s_ij = M * a_ij (2/11 取消松弛变量，约束为 w_ij - M * a_ij <= 0, 使用relu控制)
for i in range(target_width * (target_width - 1)):
    trans_restriction = np.zeros(target_width * (target_width - 1))
    trans_restriction[i] = 1
    #trans_restriction[i + target_width] = 1
    #(2/11 取消松弛变量后，A暂时无用)
    A.append(trans_restriction)
    B.append(a_ij[i]*100000)
    
A_tensor = torch.tensor(np.array(A))
B_tensor = torch.tensor(np.array(B))

In [30]:
train_covariate_df.shape[0]

78

In [31]:
average_total_loss_average = 0
loss_array = np.zeros(n * horizon_size - horizon_size + 1)

In [32]:
net = MQRNN(horizon_size, hidden_size, quantiles, columns, dropout, layer_size, b_direction, lr, batch_size,
            num_epochs, context_size, covariate_size, target_width, data_list[0].x.shape[0], A_tensor, B_tensor, price, device)

In [33]:
train_dataset = MQRNN_dataset(train_target_df, train_covariate_df, train_demand_df, train_supply_df, train_inventory_df, train_GNN_dataset,
                              horizon_size, quantiles_size)

In [34]:
a,b,c,d,e,f,g = train_dataset[1]
a1,b1,c1,d1,e1,f1,g1 = train_dataset[100]

In [35]:
print(a.size(),b.size(),c.size(),d.size(),e.size(),f.size(),len(g))

torch.Size([75, 2017]) torch.Size([75, 3075]) torch.Size([75, 3, 992]) torch.Size([75, 3, 32]) torch.Size([75, 3, 32]) torch.Size([75, 3, 32]) 75


In [36]:
start_time = time.time()
print("开始时间为:",start_time)
net.train(train_dataset,train_GNN_dataset)
end_time = time.time()
print("已用时:",end_time-start_time)

开始时间为: 1751542315.2913694
epoch_num:0
cur_series_tensor.size: torch.Size([1, 75, 2017])
GNN_dataset.size: 75
Data(x=[1616, 1616], edge_index=[2, 354470], edge_attr=[354470, 1], y=[1056], period_id=[1], milp_obj=173612.596, var_names=[1616])
需求矩阵宽度为： torch.Size([1, 75, 3, 32])
cur_demand_tensor.size torch.Size([75, 1, 3, 32])
GNN输入层格式: 75
原LSTM_Encoder输出层格式: torch.Size([75, 1, 512])


C:\Users\Yuansheng Si\Documents\Zhejiang University\易点云供需预测_0606\端到端预测\真实数据\真实数据实验\GNN_LSTM.py:95: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x)
C:\Users\Yuansheng Si\Documents\Zhejiang University\易点云供需预测_0606\端到端预测\真实数据\真实数据实验\GNN_LSTM.py:96: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(edge_index)


GNN_LSTM_Encoder输出层格式: torch.Size([75, 1, 512])
demand输出格式为： torch.Size([75, 1, 30])
demand输出格式为2： torch.Size([75, 1, 3, 10, 1])
torch.Size([75, 1, 3, 10, 1])
Ldecoder_output_PJ.size torch.Size([75, 1, 3, 10, 992])
Loss is: tensor([-167869.6875], device='cuda:0', grad_fn=<AddBackward0>)
epoch_num:1
cur_series_tensor.size: torch.Size([1, 75, 2017])
GNN_dataset.size: 75
Data(x=[1616, 1616], edge_index=[2, 354470], edge_attr=[354470, 1], y=[1056], period_id=[1], milp_obj=173612.596, var_names=[1616])
需求矩阵宽度为： torch.Size([1, 75, 3, 32])
cur_demand_tensor.size torch.Size([75, 1, 3, 32])
GNN输入层格式: 75
原LSTM_Encoder输出层格式: torch.Size([75, 1, 512])
GNN_LSTM_Encoder输出层格式: torch.Size([75, 1, 512])
demand输出格式为： torch.Size([75, 1, 30])
demand输出格式为2： torch.Size([75, 1, 3, 10, 1])
torch.Size([75, 1, 3, 10, 1])
Ldecoder_output_PJ.size torch.Size([75, 1, 3, 10, 992])
Loss is: tensor([363289.1875], device='cuda:0', grad_fn=<AddBackward0>)
epoch_num:2
cur_series_tensor.size: torch.Size([1, 75, 2017])
GNN_

KeyboardInterrupt: 

In [ ]:
test_covariate_df

In [ ]:
start_time = time.time()
print("开始时间为:",start_time)
i = 0
predict_dict1 = []
demand_dict = []
#n = int(20 / horizon_size)
#知识蒸馏时，需要对训练集数据进行一并预测，所以n为seq总长
n = int(test_target_df.shape[0]/horizon_size)
print("测试集长度为:",n)
for p in range(n * horizon_size - horizon_size + 1):
    test_target_df1 = test_target_df[p : p + horizon_size]
    test_covariate_df1 = test_covariate_df.iloc[p : p + horizon_size, :]
    predict_dict, demand_output = net.predict(train_target_df, train_covariate_df, test_covariate_df1, train_GNN_dataset, target_width, columns, A_tensor, B_tensor)
    predict_dict = np.array(list(predict_dict.values()))
    demand_output = np.array(list(demand_output.values()))
    #逆归一化
    # for i in range(quantiles_size):
    #     for j in range(horizon_size):
    #         for k in range(target_width):
    #             predict_dict[i,j,k] = predict_dict[i,j,k]*std[k] + mean[k]
    predict_1 = []
    demand_1 = []
    #取horizon第1天作为预测值
    for i in range(quantiles_size):
        predict_1.append(predict_dict[i,0])
        demand_1.append(demand_output[i,0])
    predict_1 = np.array(predict_1)
    demand_1 = np.array(demand_1)
    #print("demand_1:",demand_1)
    predict_2 = []
    demand_2 = []
    #取10个quantile的平均
    for i in range(target_width*(target_width-1)):
        #print("这是第",i,"个决策")
        sum1 = 0
        for j in range(quantiles_size):
            #print(predict_1[j,i])
            sum1 += predict_1[j,i]
        predict_2.append(sum1/10)
    demand_2.append(sum(demand_1)/10)
    predict_dict1.append(predict_2)
    demand_dict.append(demand_2)
predict_dict1 = pd.DataFrame(predict_dict1)
for i in range(len(demand_dict)):
    demand_dict[i] = int(demand_dict[i][0])
demand_dict = pd.DataFrame(demand_dict)
#predict_dict1
end_time = time.time()
print("已用时:",end_time-start_time)

In [ ]:
predict_dict1 = predict_dict1.astype(int)
predict_dict1.to_excel('output.xlsx', index=True)
column_names = data_df.columns[:target_width * (target_width - 1)]
predict_dict1.columns = column_names

In [ ]:
demand_dict

In [ ]:
predict_dict1

In [ ]:
# for i in range(len(predict_dict1)):
#     if(predict_dict1.iloc[i,0] > predict_dict1.iloc[i,1]):
#         predict_dict1.iloc[i,0] = int(predict_dict1.iloc[i,0] - predict_dict1.iloc[i,1])
#         predict_dict1.iloc[i,1] = 0
#     else:
#         predict_dict1.iloc[i,1] = int(predict_dict1.iloc[i,1] - predict_dict1.iloc[i,0])
#         predict_dict1.iloc[i,0] = 0

In [ ]:
predict_demand_arr = np.array(demand_dict)
predict_demand_arr = predict_demand_arr[-18:]

In [ ]:
len(predict_demand_arr)

In [ ]:
test_demand_df_1 = demand_df.iloc[-20:-horizon_size + 1]
test_demand_df_1 = test_demand_df_1.sum(axis=1)
test_demand_df_1=pd.DataFrame(test_demand_df_1)
test_demand_df_arr = np.array(test_demand_df_1)
print(len(test_demand_df_arr))

In [ ]:
test_demand_df_arr

In [ ]:
predict_demand_arr

In [ ]:
error_rate = 0
for i in range(len(predict_demand_arr)):
    error_rate += 1-abs(predict_demand_arr[i][0]-test_demand_df_arr[i][0])/test_demand_df_arr[i][0]
error_rate/=len(predict_demand_arr)

In [ ]:
teacher_loss = loss_result(target_width, predict_dict1, supply_df, inventory_df, demand_df, price)

In [ ]:
#学生模型测试集为20天
n = int(20 / horizon_size)
covariate_size = train_covariate_df.shape[1]
predict_dict_student = predict_dict1.iloc[horizon_size :-n * horizon_size, :]
train_target_df = train_target_df[:-horizon_size + 1]
train_covariate_df = train_covariate_df[:-horizon_size + 1]
hidden_size = 1024
lr = 7e-4
num_epochs = 100

In [ ]:
student_net = Student_Model(hidden_size, lr, batch_size, num_epochs, covariate_size, target_width, device)

In [ ]:
start_time = time.time()
print("开始时间为:",start_time)
student_net.train(A_tensor,B_tensor,train_target_df,train_covariate_df,predict_dict_student)
end_time = time.time()
print("已用时:",end_time-start_time)

In [ ]:
test_target_df = target_df.iloc[-horizon_size * n:]
test_covariate_df = covariate_df.iloc[-(n + 1) * horizon_size:-horizon_size, :]
print(test_covariate_df.shape)

In [ ]:
print(test_target_df.shape)

In [ ]:
start_time = time.time()
print("开始时间为:",start_time)
student_prediction, student_loss = student_net.predict(A_tensor,B_tensor,test_target_df,test_covariate_df)
end_time = time.time()
print("已用时:",end_time-start_time)

In [ ]:
student_prediction = pd.DataFrame(student_prediction)
student_prediction = student_prediction.astype(int)
student_prediction

In [ ]:
student_loss = loss_result(target_width, student_prediction, supply_df.iloc[-n*horizon_size:,], inventory_df.iloc[-n*horizon_size:,], demand_df.iloc[-n*horizon_size:,], price)
print(student_loss)

In [ ]:
Gurobi_dict = data_df.iloc[:-horizon_size + 1,(target_width + 2) * target_width + 2:(2 * target_width + 1) * target_width + 2]
Gurobi_dict

In [ ]:
Gurobi_loss = loss_result(target_width, Gurobi_dict, supply_df, inventory_df, demand_df, price)
print(Gurobi_loss)

In [ ]:
print("需求预测准确率为：",error_rate*100,"%")
print("求解器loss为：",Gurobi_loss)
print("老师模型loss为：",teacher_loss)
print("学生模型loss为：",student_loss)